In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import scipy
import os
import nept

from loading_data import get_data
from utils_maze import get_bin_centers, get_zones, get_trials, get_xy_idx

In [ ]:
import info.r063d7 as r063d7
import info.r063d8 as r063d8
# infos = [r063d7, r063d8]
from run import analysis_infos
infos = analysis_infos

In [ ]:
thisdir = os.getcwd()
pickle_filepath = os.path.join(thisdir, "cache", "pickled")
output_filepath = os.path.join(thisdir, "plots", "behavior-bytrajectory")
if not os.path.exists(output_filepath):
    os.makedirs(output_filepath)

In [ ]:
savefigs = False

u_colour = "#1d91c0"
shortcut_colour = "#41ab5d"
novel_colour = "#ef6548"
colours = [u_colour, shortcut_colour, novel_colour]

In [ ]:
min_samples = 50
len_min_trials = 30

maze_segments = ["u", "shortcut", "novel"]
n_sessions = 0

all_n_trials = {key: [] for key in maze_segments}
all_trial_duration = {key: [] for key in maze_segments}
all_trial_order = {key: [] for key in maze_segments}

for info in infos:
    events, position, _, _, _ = get_data(info)
    
    n_sessions += 1

    phase = info.task_times["phase3"]
    trials = get_trials(events, phase)

    u_zone, shortcut_zone, novel_zone = get_zones(info, position)

    n_trials = {key: 0 for key in maze_segments}
    trial_duration = {key: [] for key in maze_segments}
    trial_order = {key: np.zeros(len_min_trials) for key in maze_segments}

    for i, (trial_start, trial_stop) in enumerate(zip(trials.starts, trials.stops)):
        x_idxs, y_idxs = get_xy_idx(info, position.time_slice(trial_start, trial_stop))

        if np.sum(novel_zone[y_idxs, x_idxs]) > min_samples:
            n_trials["novel"] += 1
            trial_duration["novel"].append(trial_stop-trial_start)
            if i < len_min_trials:
                trial_order["novel"][i] = 1.0
        elif np.sum(shortcut_zone[y_idxs, x_idxs]) > min_samples:
            n_trials["shortcut"] += 1
            trial_duration["shortcut"].append(trial_stop-trial_start)
            if i < len_min_trials:
                trial_order["shortcut"][i] = 1.0
        elif np.sum(u_zone[y_idxs, x_idxs]) > min_samples:
            n_trials["u"] += 1
            trial_duration["u"].append(trial_stop-trial_start)
            if i < len_min_trials:
                trial_order["u"][i] = 1.0
    for maze_segment in maze_segments:
        all_n_trials[maze_segment].append(n_trials[maze_segment])
        all_trial_duration[maze_segment].extend(trial_duration[maze_segment])
        all_trial_order[maze_segment].append(trial_order[maze_segment])

In [ ]:
fig, ax = plt.subplots()
n_bars = np.arange(3)
n_trials = np.sum([np.mean(all_n_trials[maze_segment]) for maze_segment in maze_segments])
plt.bar(n_bars, [np.mean(all_n_trials[maze_segment])/n_trials for maze_segment in maze_segments], color=colours,
        yerr=[scipy.stats.sem(all_n_trials[maze_segment])/n_trials for maze_segment in maze_segments])
plt.xticks(n_bars, maze_segments)

if n_sessions == 1:
    title = "Proporiton for " + info.session_id
else:
    title = "Proporiton for all sessions"
    
plt.title(title)
plt.ylabel("Proportion of trials")
ax.spines['right'].set_visible(False)
ax.spines['top'].set_visible(False)

plt.tight_layout()

if savefigs:
    plt.savefig(os.path.join(output_filepath, title+".png"))
    plt.close()
else:
    plt.show()

In [ ]:
fig, ax = plt.subplots()
bp = plt.boxplot([all_trial_duration[maze_segment] for maze_segment in maze_segments], patch_artist=True)

## change outline color, fill color and linewidth of the boxes
for box in bp['boxes']:
    # change outline color
    box.set(color='#543005', linewidth=2)
    # change fill color
    box.set(facecolor = '#35978f')

## change color and linewidth of the whiskers
for whisker in bp['whiskers']:
    whisker.set(color='#543005', linewidth=2)

## change color and linewidth of the caps
for cap in bp['caps']:
    cap.set(color='#543005', linewidth=2)

## change color and linewidth of the medians
for median in bp['medians']:
    median.set(color='#80cdc1', linewidth=2)

## change the style of fliers and their fill
for flier in bp['fliers']:
    flier.set(marker='o', color='#e7298a', alpha=0.5)
    
ax.set_xticklabels(maze_segments)
ax.spines['right'].set_visible(False)
ax.spines['top'].set_visible(False)

if n_sessions == 1:
    title = "Trial duration for " + info.session_id
else:
    title = "Trial duration for all sessions"
    
plt.title(title)
plt.ylim(0, 150)

plt.tight_layout()
if savefigs:
    plt.savefig(os.path.join(output_filepath, title+".png"))
    plt.close()
else:
    plt.show()

In [ ]:
trials = np.arange(30)

fig, ax = plt.subplots(figsize=(7,5))

u_mean = np.mean(all_trial_order["u"], axis=0)
u_sem = scipy.stats.sem(all_trial_order["u"], axis=0)
plt.plot(u_mean, "b")
ax.fill_between(trials, u_mean - u_sem,
                        u_mean + u_sem,
                        color=u_colour, interpolate=True, alpha=0.2)

shortcut_mean = np.mean(all_trial_order["shortcut"], axis=0)
shortcut_sem = scipy.stats.sem(all_trial_order["shortcut"], axis=0)
plt.plot(shortcut_mean, "g")
ax.fill_between(trials, shortcut_mean - shortcut_sem,
                        shortcut_mean + shortcut_sem,
                        color=shortcut_colour, interpolate=True, alpha=0.2)

novel_mean = np.mean(all_trial_order["novel"], axis=0)
novel_sem = scipy.stats.sem(all_trial_order["novel"], axis=0)
plt.plot(novel_mean, "r")
ax.fill_between(trials, novel_mean - novel_sem,
                        novel_mean + novel_sem,
                        color=novel_colour, interpolate=True, alpha=0.2)

ax.legend(maze_segments, bbox_to_anchor=(1.2, 1.0))
ax.spines['right'].set_visible(False)
ax.spines['top'].set_visible(False)

if n_sessions == 1:
    title = "Choice over time for " + info.session_id
else:
    title = "Choice over time for all sessions"
plt.title(title)

plt.tight_layout()
if savefigs:
    plt.savefig(os.path.join(output_filepath, title+".png"))
    plt.close()
else:
    plt.show()

In [ ]:
trials = get_trials(events, phase)
for trial_start, trial_stop in zip(trials.starts[:10], trials.stops[:10]):

    tt = position.time_slice(trial_start, trial_stop)

    plt.plot(position.x, position.y, "y.")
    plt.plot(tt.x, tt.y, "k.", ms=4)
    plt.show()

In [ ]:
7. + 9.97 + 15. + 9.09

In [ ]:
60-41.06